# Handwritten Digit Generator (DCGAN on MNIST)
This notebook trains a conditional GAN to generate handwritten digits using the MNIST dataset.

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Settings
latent_dim = 100
num_epochs = 10
batch_size = 128
learning_rate = 0.0002
image_size = 28
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])
mnist = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
data_loader = DataLoader(mnist, batch_size=batch_size, shuffle=True)

In [ ]:
# Generator
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Linear(latent_dim + 10, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, image_size * image_size),
            nn.Tanh()
        )

    def forward(self, z, labels):
        label_input = torch.nn.functional.one_hot(labels, 10).float()
        x = torch.cat((z, label_input), dim=1)
        out = self.main(x)
        return out.view(-1, 1, image_size, image_size)

In [ ]:
# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Linear(image_size * image_size + 10, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x, labels):
        x = x.view(-1, image_size * image_size)
        label_input = torch.nn.functional.one_hot(labels, 10).float()
        x = torch.cat((x, label_input), dim=1)
        return self.main(x)

In [ ]:
# Initialize
G = Generator().to(device)
D = Discriminator().to(device)
criterion = nn.BCELoss()
optimizer_G = torch.optim.Adam(G.parameters(), lr=learning_rate, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(D.parameters(), lr=learning_rate, betas=(0.5, 0.999))

In [ ]:
# Train
for epoch in range(num_epochs):
    for i, (real_imgs, labels) in enumerate(data_loader):
        batch_size = real_imgs.size(0)
        real_imgs, labels = real_imgs.to(device), labels.to(device)

        valid = torch.ones(batch_size, 1, device=device)
        fake = torch.zeros(batch_size, 1, device=device)

        # Train Generator
        z = torch.randn(batch_size, latent_dim, device=device)
        gen_imgs = G(z, labels)
        g_loss = criterion(D(gen_imgs, labels), valid)
        optimizer_G.zero_grad()
        g_loss.backward()
        optimizer_G.step()

        # Train Discriminator
        real_loss = criterion(D(real_imgs, labels), valid)
        fake_loss = criterion(D(gen_imgs.detach(), labels), fake)
        d_loss = (real_loss + fake_loss) / 2
        optimizer_D.zero_grad()
        d_loss.backward()
        optimizer_D.step()

    print(f"[Epoch {epoch+1}/{num_epochs}] D loss: {d_loss.item():.4f} | G loss: {g_loss.item():.4f}")

In [ ]:
# Save model
torch.save(G.state_dict(), "generator.pth")
print("Generator model saved as 'generator.pth'")